In [1]:
import os
import json
import random
import re
import pandas as pd

jokes_filepath = os.path.join('..', 'data', 'jokes.csv')
prep_outpath = os.path.join('..', 'data', 'prep', 'gpt2.txt')

## Load the Model

In [2]:
!git clone -b small https://github.com/Tenoke/gpt-2.git

fatal: destination path 'gpt-2' already exists and is not an empty directory.


In [2]:
cd gpt-2

C:\Users\kzvda\Desktop\Project\gpt-2


In [4]:
# Download the model

# Linux
# !sh download_model.sh 117M

# Windows
!download_model.sh 117M

## Load data

In [11]:
import numpy as np
with np.load('../data/prep/gpt2.txt.npz') as data:
    print(data['arr_0'])

[    2 48443 14629    14  8800    14 24330 21015    18   198     2 29566
    25   198     2   220   350    56  4221  1340 34219    28 10677 24457
   268  8189    13  9078  1279  7753    91 34945    91  4743   672    29
  1220  6978    14  1462    14 22915    13 37659    89   198     2   220
   350    56  4221  1340 34219    28 10677 24457 27432  1377 19608   292
   316  1220  6978    14  1462    14 22915    13 37659    89   198   198
 11748  2046   198 11748 33918   198 11748 28686   198 11748   299 32152
   355 45941   198 11748 11192   273 11125   355 48700   198 11748  4738
   198 11748   640   198 11748   256    80 36020   198 11748 15095   198
   198 11748  2207 12342   628   198  4299  3440    62 19608   292   316
     7 12685    11  3108  2599   198   220   220   220 13532   796 17635
   198   220   220   220   611 28686    13  6978    13  4468   576     7
  6978  2599   198   220   220   220   220   220   220   220  1303 17427
  2393   198   220   220   220   220   220   220   

In [5]:
data = pd.read_csv(jokes_filepath)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38269 entries, 0 to 38268
Data columns (total 3 columns):
ID          38269 non-null int64
Question    38269 non-null object
Answer      38269 non-null object
dtypes: int64(1), object(2)
memory usage: 897.1+ KB


In [6]:
def fix_encoding(s):
#     return re.sub('[\xc2-\xf4][\x80-\xbf]+',lambda m: m.group(0).encode('latin1').decode('utf8'),s)
    return s.encode('latin1', 'ignore').decode('utf8', 'ignore')

text_corpus = ''
for _, question, answer in data.values:
    text_corpus += fix_encoding(f'q: {question}\na: {answer}\n')
text_corpus += '\n\n'

In [7]:
# Create the corresponding folder (if needed)
os.makedirs(os.path.dirname(prep_outpath), exist_ok=True)
with open(prep_outpath, 'w') as out_file:
    out_file.write(text_corpus)

In [ ]:
# os.system(f'python src/encode.py --in-text {prep_outpath} --out-npz {prep_outpath}.npz')

!..\pythonpath.bat src encode.py "..\data\prep\gpt2.txt" "..\data\prep\gpt2.txt.npz"

## Train

In [ ]:
!..\pythonpath.bat src train.py --dataset ..\data\prep\gpt2.txt.npz --sample_every=250 --learning_rate=0.0001 --stop_after=251

In [ ]:
!..\pythonpath.bat src train.py --dataset ..\data\prep\gpt2.txt.npz --sample_every=250 --learning_rate=0.001 --stop_after=751

In [ ]:
!..\pythonpath.bat src train.py --dataset ..\data\prep\gpt2.txt.npz --sample_every=250 --learning_rate=0.0001 --beta=0.95 --stop_after=1251

In [ ]:
# !cp -r /content/gpt-2/checkpoint/run1/* /content/gpt-2/models/117M/
from distutils.dir_util import copy_tree
copy_tree('checkpoint/run1', 'models/run1')

## Eval

In [ ]:
!python src/interactive_conditional_samples.py --model_name run1 --top_k 5 --temperature 0.9
# q: Did you hear about the Native American man that drank 200 cups of tea?